# Chapter 11: GANs - MINST

In [1]:
!nvidia-smi

Tue Jun 14 03:46:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [13]:
%cd '/content/gdrive/MyDrive/LDS8_K275_ONLINE_DinhVietTrung/LDS8_K275_DinhVietTrung_Cuoi_ky/food_bonus'

/content/gdrive/MyDrive/LDS8_K275_ONLINE_DinhVietTrung/LDS8_K275_DinhVietTrung_Cuoi_ky/food_bonus


In [14]:
# pip install tqdm
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf

In [15]:
from tensorflow.keras.layers import Input
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Reshape, Dense, Dropout, Flatten, LeakyReLU
from tensorflow.keras.layers import Convolution2D, UpSampling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K
from tensorflow.keras import initializers

In [16]:
from tensorflow.keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization
from tensorflow.keras import applications
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image

In [17]:
# Deterministic output.
# Tired of seeing the same results every time? Remove the line below.
# np.random.seed(1000)

# The results are a little better when the dimensionality of the random vector is only 10.
# The dimensionality has been left at 100 for consistency with other GAN implementations.
random_dim = 100

In [58]:
img_height = 128
img_width = 128
batchSize = 128

data_dir = '/content/gdrive/MyDrive/LDS8_K275_ONLINE_DinhVietTrung/LDS8_K275_DinhVietTrung_Cuoi_ky/data_bonus/pho/pho'

In [59]:
# !unzip ../data/pho.zip -d ../data/pho


In [60]:
%pwd

'/content/gdrive/MyDrive/LDS8_K275_ONLINE_DinhVietTrung/LDS8_K275_DinhVietTrung_Cuoi_ky/food_bonus'

In [61]:
import glob

food = glob.glob('/content/gdrive/MyDrive/LDS8_K275_ONLINE_DinhVietTrung/LDS8_K275_DinhVietTrung_Cuoi_ky/data_bonus/pho/pho/*')

data = []
labels = []

for i in food:   
    image=tf.keras.preprocessing.image.load_img(i, color_mode='rgb', target_size = (img_width, img_height, 3))
    image=np.array(image)
    data.append(image)
    labels.append(0)

data = np.array(data)
labels = np.array(labels)



In [62]:
data.shape

(1000, 128, 128, 3)

In [63]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [64]:
X_train.shape

(800, 128, 128, 3)

In [65]:
X_train[:1]

array([[[[ 11,  12,   7],
         [  9,  10,   5],
         [  1,   3,   0],
         ...,
         [ 72,  56,  40],
         [ 74,  59,  38],
         [103,  89,  63]],

        [[  6,   7,   0],
         [  3,   4,   0],
         [ 13,  16,   7],
         ...,
         [ 58,  39,  25],
         [ 58,  42,  27],
         [ 69,  56,  39]],

        [[  2,   3,   0],
         [ 23,  24,  18],
         [  5,   8,   1],
         ...,
         [ 55,  23,  10],
         [ 36,  10,   0],
         [ 57,  34,  20]],

        ...,

        [[138, 114,  42],
         [133, 104,  44],
         [133, 104,  44],
         ...,
         [171, 163, 150],
         [170, 164, 150],
         [168, 162, 146]],

        [[138, 111,  40],
         [128,  99,  39],
         [137, 111,  52],
         ...,
         [169, 162, 146],
         [168, 162, 146],
         [168, 163, 143]],

        [[140, 115,  49],
         [140, 113,  58],
         [128, 103,  47],
         ...,
         [169, 162, 146],
        

In [66]:
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(X_train.shape[0], img_height*img_height*3)

In [67]:
X_train[:3]

array([[-0.9137255 , -0.90588236, -0.94509804, ...,  0.31764707,
         0.2784314 ,  0.10588235],
       [ 0.18431373, -0.01960784, -0.23137255, ..., -0.9137255 ,
        -0.41960785, -0.49019608],
       [ 0.8117647 ,  0.6862745 ,  0.48235294, ...,  0.7490196 ,
         0.5294118 ,  0.13725491]], dtype=float32)

In [69]:
target_size = img_width*img_height*3
target_size

49152

In [84]:
# You will use the Adam optimizer
def get_optimizer():
    return Adam(learning_rate=0.0001, beta_1=0.5)

In [85]:
def get_generator(optimizer):
    # LeakyReLU: ReLU mà có giá trị âm á
    generator = Sequential()
    generator.add(Dense(256, 
                        input_dim=random_dim, 
                        kernel_initializer=initializers.RandomNormal(stddev=0.02))) #https://www.tensorflow.org/api_docs/python/tf/keras/initializers/RandomNormal
    generator.add(LeakyReLU(0.2)) #https://en.wikipedia.org/wiki/Rectifier_(neural_networks)#Leaky_ReLU, Leaky ReLUs allow a small, positive gradient when the unit is not active
    generator.add(BatchNormalization())

    generator.add(Dense(512))
    generator.add(LeakyReLU(0.2))
    generator.add(BatchNormalization())

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    generator.add(Dropout(0.3))
    generator.add(BatchNormalization())

    generator.add(Dense(1024))
    generator.add(LeakyReLU(0.2))
    generator.add(Dropout(0.3))
    generator.add(BatchNormalization())

    generator.add(Dense(1024*3))
    generator.add(LeakyReLU(0.2))
    generator.add(Dropout(0.2))
    generator.add(BatchNormalization())

    generator.add(Dense(1024*3))
    generator.add(LeakyReLU(0.2))
    generator.add(Dropout(0.2))
    generator.add(BatchNormalization())

    generator.add(Dense(target_size, activation='tanh'))
    generator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return generator

In [91]:
def get_discriminator(optimizer):
    discriminator = Sequential()
    target_size = img_width*img_height*3

    discriminator.add(Dense(1024, 
                            input_dim=target_size, 
                            kernel_initializer=initializers.RandomNormal(stddev=0.02)))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))

    discriminator.add(Dense(1024))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.3))
    discriminator.add(BatchNormalization())

    discriminator.add(Dense(1024*3))
    discriminator.add(LeakyReLU(0.2))
    discriminator.add(Dropout(0.2))
    discriminator.add(BatchNormalization())

    discriminator.add(Dense(1024*3))
    discriminator.add(LeakyReLU(0.3))
    discriminator.add(Dropout(0.2))
    discriminator.add(BatchNormalization())

    discriminator.add(Dense(1024*3))
    discriminator.add(LeakyReLU(0.4))
    discriminator.add(Dropout(0.2))
    discriminator.add(BatchNormalization())



    discriminator.add(Dense(1, activation='sigmoid'))
    discriminator.compile(loss='binary_crossentropy', optimizer=optimizer)

    return discriminator

In [92]:
def get_gan_network(discriminator, random_dim, generator, optimizer):
    # We initially set trainable to False 
    # since we only want to train either the generator or discriminator at a time
    discriminator.trainable = False
    # gan input (noise) will be 100-dimensional vectors
    gan_input = Input(shape=(random_dim,)) 
    # the output of the generator (an image)
    x = generator(gan_input)
    # get the output of the discriminator (probability if the image is real or not)
    gan_output = discriminator(x)
    gan = Model(inputs=gan_input, outputs=gan_output)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer)
    return gan

In [108]:
# Create a wall of generated MNIST images
def plot_generated_images(epoch, generator, examples=4, dim=(2, 2), figsize=(40, 40)):
    noise = np.random.normal(0, 1, size=[examples, random_dim]) # random_dim = 100
    generated_images = generator.predict(noise)
    generated_images = generated_images.reshape(examples, img_width, img_height, 3)

    plt.figure(figsize=figsize)
    for i in range(generated_images.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generated_images[i], interpolation='nearest', cmap='gray_r')
        plt.axis('off')        
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)

In [114]:
def train(epochs, batch_size,  x_train, y_train, x_test, y_test):

    batch_count = x_train.shape[0] / batch_size
    # Build our GAN netowrk
    adam = get_optimizer()
    generator = get_generator(adam)
    discriminator = get_discriminator(adam)
    gan = get_gan_network(discriminator, random_dim, generator, adam)

    for e in range(1, epochs+1):
        if e % 100 == 0:
            print('-'*15, 'Epoch %d' % e, '-'*15)
        # tao thanh process bar: https://tqdm.github.io/
        for _ in tqdm(range(int(batch_count))): 
            # Get a random set of input noise and images
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            image_batch = x_train[np.random.randint(0, x_train.shape[0], size=batch_size)]

            # Generate fake MNIST images
            generated_images = generator.predict(noise)
            X = np.concatenate([image_batch, generated_images])

            # Labels for generated and real data
            y_dis = np.zeros(2*batch_size) # 
            # One-sided label smoothing
            y_dis[:batch_size] = 0.9

            # Train discriminator
            discriminator.trainable = True
            discriminator.train_on_batch(X, y_dis)

            # Train generator
            noise = np.random.normal(0, 1, size=[batch_size, random_dim])
            y_gen = np.ones(batch_size)
            discriminator.trainable = False
            gan.train_on_batch(noise, y_gen)

        if e == 1 or e % 1000 == 0:
            plot_generated_images(e, generator)
        # in ket qua ra file: # save_generated_images(e, generator)

In [115]:
train(20000, 128, X_train, y_train, X_test, y_test) # chay epoch = 300, batch = 128

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
'''
Có vẻ như sau 20000 epouch mà hình vẫn chưa dc tốt lắm nhỉ, do:
- lượng data train khá ít (chỉ khoảng 1000 hình)
- hình màu
- ...
'''